In [1]:
import numpy as np 
import pandas as pd
import os 
import sys
from distutils.dir_util import copy_tree

# go to the directory where utility functions are stored
os.chdir(r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling')
from BuildingModelClass import BuildingModel
from utils import *
import subprocess
import json
from json import JSONEncoder
from os import spawnle
import win32api

In [3]:
# Utility function directory 
UtilDirectory = r'C:\Users\User\Desktop\WoodFrameBuildingTool\Python Tool Win\ModelAssembling'
# Base directory is the main directory that models, model inputs and utility directory stores
BaseDirectory = UtilDirectory.replace("ModelAssembling", "")
# Model directory is where you want to store your model
ModelDirectory = BaseDirectory + '\BuildingModels'
# DB directory is where you store Database.csv (for steel section)
DBDirecctory = UtilDirectory

# If there is no model directory, create one
if os.path.isdir(ModelDirectory) != True:
    os.chdir(BaseDirectory)
    os.mkdir('BuildingModels')
    
# Read in building name 
# Make sure the building name is consistent with the input folders
os.chdir(UtilDirectory)
with open('BuildingName.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')

# Generate eigen analysis, pushover analysis and dynamic analysis models for each one in the building list
for i in range(len(BuildingList)):
    InfoDirectory = BaseDirectory + '\BuildingInfo\%s'%BuildingList[i]
    ModelClass = BuildingModel(BuildingList[i], InfoDirectory)

    # alternatively, one can choose to read inputs from json 
    ModelClass.read_in_txt_inputs(BuildingList[i], InfoDirectory)

    if os.path.isdir(ModelDirectory+'\%s'%BuildingList[i]) != True:
        os.chdir(ModelDirectory)
        os.mkdir('%s'%BuildingList[i])
        
    os.chdir(ModelDirectory+'/%s'%BuildingList[i])
    period = generateModalAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory)
    # Turn off RunPushoverSwitch to speed up the model creation 
    generatePushoverAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory,
                               GenerateModelSwitch = True, RunPushoverSwitch = False)
    generateDynamicAnalysisModel(ModelClass.ID, ModelClass, BaseDirectory, DBDirecctory, period,
                               GenerateModelSwitch = True)
    print(period)


[0.3522614619208698, 0.25412715353010734, 0.229149425056406, 0.10394190316450584]
[0.3566760968107755, 0.26246152581949644, 0.229493709707616, 0.10406880260602897]


In [13]:
def SetupSingleGMDyamaicAnalysis(Scale_Sa_GM, GM_Num, No_GM, period, scale, pair_id, GM_id):
    os.chdir(DynamicDirectory)
    fin = open('RunDynamic.tcl', 'rt')
    fout = open('RunDynamic_Single.tcl', 'wt')

    filedata = fin.read('*Scale_Sa_GM*', Scale_Sa_GM)
    filedata = filedata.replace('*GMset_Num*', GM_Num)
    filedata = filedata.replace('*thirdModePeriod*', str(period[2]))
    filedata = filedata.replace('*GM_folder_name*', "%s"%GM_folder)
    filedata = filedata.replace('*NumberOfGroundMotions*', str(No_GM))
    filedata = filedata.replace('*Pairing_ID*', str(pair_id))
    filedata = filedata.replace('*GM_pair_ID*', str(GM_id))
    filedata = filedata.replace('*AllScales*', str(100))


    fout.write(filedata)
    fin.close()
    fout.close()

In [15]:
SetupSingleGMDyamaicAnalysis(r'C:/Users/User/Desktop/WoodFrameBuildingTool/Python Tool Win/BuildingModels/Building_T1-60x30-S2-GS_Ordinance/DynamicAnalysis', r'C:/Users/User/Desktop/WoodFrameBuildingTool/Python Tool Win/BuildingModels/GM_sets/FEMA44-FarFault', 44, period, 100, 1, 1)